---
jupyter: Python3
date: 2022-10-25
title: Bigquery_7일 연속 미접속 시작일 쉽게 추출하기
toc: true
categories: Bigquery
author: limyj0708
comments:
  giscus: 
    repo: quarto-dev/quarto-docs
---

# [Bigquery] 7일 연속 미접속 시작일 쉽게 추출하기

- 7일 연속 미접속 유저를 이탈 유저라고 하자.
- 최초의 7일 연속 미접속 시작일을 구해야, 이탈일을 알 수 있다.
- 기본 아이디어는 아래와 같다.
  - 접속한 날은 1, 접속하지 않은 날은 0으로 세팅한 정수 배열 생성
  - array_to_string으로 정수 배열을 문자열로 변환
  - `STRPOS("대상 문자열", '0000000')` 으로 0이 연속 7개가 등장하는 최초의 위치 찾음

```SQL
DECLARE date_array_base ARRAY<DATE>;
DECLARE power_array_base ARRAY<INT64>;
SET date_array_base = GENERATE_DATE_ARRAY(DATE('2022-08-16'), DATE('2022-10-23'), INTERVAL 1 DAY);
SET power_array_base = GENERATE_ARRAY(0, 340000, 20000);

with abuse_list as ( --어뷰징 유저 목록
  SELECT accountId
  FROM `___`
  WHERE acc_buy_krw IS NULL OR acc_buy_krw < power(10,5)

), raw_sum_sales as ( #누적과금액
  SELECT accountId, sum(won) as total_sales
  FROM `___`
  WHERE b_date < DATE(2022, 10, 23)
    AND accountId NOT IN (SELECT accountId FROM abuse_list)
  group by accountId

), max_power_daily as ( --일자별 누적 최고전투력
  SELECT b_date, accountId, max_power
  FROM `___`
  WHERE accountId NOT IN (SELECT accountId FROM abuse_list)

), acc_created_date as ( --계정 생성일
  SELECT accountId, EXTRACT(DATE FROM created_date) AS c_date
  FROM `___`
  WHERE b_date = DATE(2022, 10, 23)
    AND accountId NOT IN (SELECT accountId FROM abuse_list)

), total_logs_score_connect AS ( --로비 커넥트 로그
  SELECT * FROM (
    SELECT EXTRACT(DATE FROM date_time) AS b_date, account_id
    FROM `___`
  ) WHERE b_date > DATE(2022, 08, 15)
    AND account_id NOT IN (SELECT accountId FROM abuse_list)

), account_id_connect_date_array AS ( --유저 별 접속일자 array 만들고, c_date(계정 생성일) 붙임
  SELECT account_id, date_array, acd.c_date
  FROM (
    SELECT account_id, ARRAY_AGG(b_date ORDER BY b_date) AS date_array
    FROM (
      SELECT DISTINCT b_date, account_id
      FROM total_logs_score_connect
    )
    GROUP BY account_id
  ) base
  LEFT JOIN acc_created_date acd ON acd.accountId = base.account_id

), connect_date_max_power AS ( --접속한 날의 누적 최고전투력
  SELECT base.account_id, days, mpd.max_power
  FROM (
    SELECT account_id, days
    FROM account_id_connect_date_array, UNNEST(date_array) AS days
    WHERE ARRAY_LENGTH(date_array) > 14 --14일 초과 접속한 유저 대상
  ) base
  LEFT JOIN max_power_daily mpd ON (mpd.accountId = base.account_id AND mpd.b_date = base.days)
  WHERE days < DATE(2022, 10, 24)
  ORDER BY account_id, days

), add_gap_day_3_avg_last AS ( --접속일 기준, 3일 전투력 이동평균과 3일 전투력 이동평균의 일자별 변화량 계산
  SELECT account_id, days, max_power, day_3_avg
    ,(day_3_avg - LAG(day_3_avg) over(partition by account_id order by days)) AS gap_day_3_avg_last
  FROM (
    SELECT account_id, days, max_power
      ,avg(max_power) over(partition by account_id order by days rows between 3 preceding and current row) as day_3_avg
    FROM connect_date_max_power
  )
  ORDER BY account_id, days

), get_first_stag_start_date AS ( --최초로 4접속일 연속 전투력이 같은 날
  SELECT account_id, days, max_power, day_3_avg, gap_day_3_avg_last
  FROM (
    SELECT account_id, days, max_power, day_3_avg, gap_day_3_avg_last
      ,ROW_NUMBER() OVER(PARTITION BY account_id ORDER BY days) num
    FROM (
      SELECT account_id, days, max_power, day_3_avg, gap_day_3_avg_last
      FROM add_gap_day_3_avg_last
      WHERE gap_day_3_avg_last = 0
    )
  ) WHERE num = 1
  ORDER BY account_id, days
  
), add_connect_check_array AS (
  SELECT cda.account_id, ssd.days
    ,(
      SELECT ARRAY (
        SELECT
          CASE
            WHEN day < ssd.days THEN "3" --최초 4접속일 연속 전투력 같은 날 보다 이전 날짜는 3 
            WHEN day IN UNNEST(date_array) THEN "1" --접속일은 1
            WHEN day < c_date THEN "2" --계정 생성일보다 이전날짜는 2
            ELSE "0" --접속 안했으면 0
            END AS element
        FROM UNNEST(date_array_base) as day
      )
    ) AS connect_check_array
    ,date_array
  FROM account_id_connect_date_array cda
  LEFT JOIN get_first_stag_start_date ssd ON ssd.account_id = cda.account_id
  WHERE ARRAY_LENGTH(date_array) > 14 --14일 초과 접속한 유저 대상

), add_leaving_position AS (
  SELECT account_id, array_string, STRPOS(array_string, '0000000') AS leaving_pos --7일 연속 최초 미접속 확인부분
  FROM (
    SELECT account_id, connect_check_array, array_to_string(connect_check_array, '') AS array_string
    FROM add_connect_check_array
  )

), get_leaving_date AS (
  SELECT account_id, DATE_ADD(DATE(2022,08,16), INTERVAL leaving_pos-1 DAY) AS leaving_date --8/16이 위치 1이기 때문에, leaving_pos에서 1을 빼줌
  FROM add_leaving_position
  WHERE leaving_pos != 0 --7일 연속 미접속이 없는 계정은 제외

)

SELECT ssd.account_id, days, max_power, day_3_avg, gap_day_3_avg_last, gld.leaving_date, DATE_DIFF(gld.leaving_date, days, DAY) AS gap,
  CASE
    WHEN max_power BETWEEN power_array_base[OFFSET(0)] AND power_array_base[OFFSET(1)] THEN "01. 0 ~ 20000"
    WHEN max_power BETWEEN power_array_base[OFFSET(1)]+1 AND power_array_base[OFFSET(2)] THEN "02. 20001 ~ 40000"
    WHEN max_power BETWEEN power_array_base[OFFSET(2)]+1 AND power_array_base[OFFSET(3)] THEN "03. 40001 ~ 60000"
    WHEN max_power BETWEEN power_array_base[OFFSET(3)]+1 AND power_array_base[OFFSET(4)] THEN "04. 60001 ~ 80000"
    WHEN max_power BETWEEN power_array_base[OFFSET(4)]+1 AND power_array_base[OFFSET(5)] THEN "05. 80001 ~ 100000"
    WHEN max_power BETWEEN power_array_base[OFFSET(5)]+1 AND power_array_base[OFFSET(6)] THEN "06. 100001 ~ 120000"
    WHEN max_power BETWEEN power_array_base[OFFSET(6)]+1 AND power_array_base[OFFSET(7)] THEN "07. 120001 ~ 140000"
    WHEN max_power BETWEEN power_array_base[OFFSET(7)]+1 AND power_array_base[OFFSET(8)] THEN "08. 140001 ~ 160000"
    WHEN max_power BETWEEN power_array_base[OFFSET(8)]+1 AND power_array_base[OFFSET(9)] THEN "09. 160001 ~ 180000"
    WHEN max_power BETWEEN power_array_base[OFFSET(9)]+1 AND power_array_base[OFFSET(10)] THEN "10. 180001 ~ 200000"
    WHEN max_power BETWEEN power_array_base[OFFSET(10)]+1 AND power_array_base[OFFSET(11)] THEN "11. 200001 ~ 220000"
    WHEN max_power BETWEEN power_array_base[OFFSET(11)]+1 AND power_array_base[OFFSET(12)] THEN "12. 220001 ~ 240000"
    WHEN max_power BETWEEN power_array_base[OFFSET(12)]+1 AND power_array_base[OFFSET(13)] THEN "13. 240001 ~ 260000"
    WHEN max_power BETWEEN power_array_base[OFFSET(13)]+1 AND power_array_base[OFFSET(14)] THEN "14. 260001 ~ 280000"
    WHEN max_power BETWEEN power_array_base[OFFSET(14)]+1 AND power_array_base[OFFSET(15)] THEN "15. 280001 ~ 300000"
    WHEN max_power BETWEEN power_array_base[OFFSET(15)]+1 AND power_array_base[OFFSET(16)] THEN "16. 300001 ~ 320000"
    WHEN max_power BETWEEN power_array_base[OFFSET(16)]+1 AND power_array_base[OFFSET(17)] THEN "17. 320001 ~ 340000"
    WHEN max_power > power_array_base[OFFSET(17)] THEN "18. 340001 ~ "
    ELSE "None"
  END AS end_power_cate
  ,CASE
    WHEN total_sales IS NULL THEN '1. 무과금 (0원)'
    WHEN total_sales > 0 AND total_sales < 1300 THEN '2. 베이직 (1200원)'
    WHEN total_sales BETWEEN 1301 AND 22000 THEN '3. 소과금' 
    WHEN total_sales BETWEEN 22001 AND 60000 THEN '4. 중과금'
    WHEN total_sales BETWEEN 60001 AND 240000 THEN '5. 중고과금'
    WHEN total_sales BETWEEN 240001 AND 500000 THEN '6. 고과금'
    WHEN total_sales BETWEEN 500001 AND 1000000 THEN '7. VIP'
    WHEN total_sales BETWEEN 1000001 AND 10000000 THEN '8. VVIP'
    WHEN total_sales > 10000000 THEN '9. 고래유저'
    ELSE "None"
  END AS total_sales_cate
FROM get_first_stag_start_date ssd
LEFT JOIN get_leaving_date gld ON gld.account_id = ssd.account_id
LEFT JOIN raw_sum_sales rss ON rss.accountId = ssd.account_id
WHERE leaving_date IS NOT NULL
```